In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/homework-1-dataset/sample_submission.csv
/kaggle/input/homework-1-dataset/data_description.txt
/kaggle/input/homework-1-dataset/train.csv
/kaggle/input/homework-1-dataset/test.csv


In [30]:
df = pd.read_csv('/kaggle/input/homework-1-dataset/train.csv')

# Train/Test Split

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df,test_size=0.2, random_state=42)

In [32]:
X_train.shape, X_test.shape

((1168, 81), (292, 81))

In [33]:
null_ratio = X_train.isna().mean()
too_much_null_columns = null_ratio[null_ratio >= 0.5]
print(too_much_null_columns)
X_train.drop(columns = too_much_null_columns.index, inplace = True)
X_test.drop(columns = too_much_null_columns.index, inplace = True)
X_train.shape, X_test.shape

Alley          0.936644
MasVnrType     0.584760
PoolQC         0.994863
Fence          0.800514
MiscFeature    0.960616
dtype: float64


((1168, 76), (292, 76))

In [34]:
for col in X_train.select_dtypes(include=['number']):
    change_null_to_this = X_train[col].mean()
    X_train.fillna(change_null_to_this, inplace=True)
    X_test.fillna(change_null_to_this, inplace=True)
X_train.isna().mean()

Id               0.0
MSSubClass       0.0
MSZoning         0.0
LotFrontage      0.0
LotArea          0.0
                ... 
MoSold           0.0
YrSold           0.0
SaleType         0.0
SaleCondition    0.0
SalePrice        0.0
Length: 76, dtype: float64

In [35]:
for col in X_train.select_dtypes(include=['object']):
    change_null_to_this = X_train[col].mode()
    X_train.fillna(change_null_to_this, inplace=True)
    X_test.fillna(change_null_to_this, inplace=True)
X_train.isna().mean()

Id               0.0
MSSubClass       0.0
MSZoning         0.0
LotFrontage      0.0
LotArea          0.0
                ... 
MoSold           0.0
YrSold           0.0
SaleType         0.0
SaleCondition    0.0
SalePrice        0.0
Length: 76, dtype: float64

In [36]:
from sklearn.preprocessing import OrdinalEncoder

for col in X_train.select_dtypes(include=['object']):
    encoder = OrdinalEncoder()
    
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)
    
    X_train[col] = encoder.fit_transform(X_train[[col]])
    X_test[col] = encoder.fit_transform(X_test[[col]])

In [37]:
# X_train = X_train.select_dtypes(exclude=['object'])
# X_test = X_test.select_dtypes(exclude=['object'])

X_train.shape, X_test.shape

((1168, 76), (292, 76))

In [38]:
Y_train = X_train.pop('SalePrice')
Y_test = X_test.pop('SalePrice')

Y_train.shape, Y_test.shape

((1168,), (292,))

# Training

In [39]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X_train, Y_train)

In [40]:
prob_train = model.predict(X_train)
prob_test = model.predict(X_test)

In [41]:
from sklearn.metrics import *

r2 = r2_score(Y_train, prob_train)
print(f"R2 score: {r2}")
rmse = mean_squared_error(Y_train, prob_train, squared=False)
print(f"RMSE score: {rmse}")
mae = mean_absolute_error(Y_train, prob_train)
print(f"MAE score: {mae}")

R2 score: 0.8438433059767971
RMSE score: 30518.919847199715
MAE score: 18601.19273208766


In [42]:
r2 = r2_score(Y_test, prob_test)
print(f"R2 score: {r2}")
rmse = mean_squared_error(Y_test, prob_test, squared=False)
print(f"RMSE score: {rmse}")
mae = mean_absolute_error(Y_test, prob_test)
print(f"MAE score: {mae}")

R2 score: 0.8549587591654018
RMSE score: 33354.36449247022
MAE score: 21093.976980656425


# Train using Ridge

In [102]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=2000.0).fit(X_train, Y_train)

In [103]:
prob_train = model.predict(X_train)
prob_test = model.predict(X_test)

In [104]:
from sklearn.metrics import *

r2 = r2_score(Y_train, prob_train)
print(f"R2 score: {r2}")
rmse = mean_squared_error(Y_train, prob_train, squared=False)
print(f"RMSE score: {rmse}")
mae = mean_absolute_error(Y_train, prob_train)
print(f"MAE score: {mae}")

print()

r2 = r2_score(Y_test, prob_test)
print(f"R2 score: {r2}")
rmse = mean_squared_error(Y_test, prob_test, squared=False)
print(f"RMSE score: {rmse}")
mae = mean_absolute_error(Y_test, prob_test)
print(f"MAE score: {mae}")

R2 score: 0.8001737853313143
RMSE score: 34523.52132373247
MAE score: 19690.594356155318

R2 score: 0.8308806976767621
RMSE score: 36016.66581946661
MAE score: 20925.446425622315


In [95]:
!pip install mlflow dagshub

In [96]:
import mlflow

In [105]:
import dagshub
dagshub.init(repo_owner='Givi-Modebadze', repo_name='my-first-repo', mlflow=True)

Initialized MLflow to track repo "Givi-Modebadze/my-first-repo"

Repository Givi-Modebadze/my-first-repo initialized!

In [106]:
experiment = "my first experimnet"
run_name = "Ridge with cleaned features"

In [107]:
mlflow.set_experiment(experiment)

<Experiment: artifact_location='mlflow-artifacts:/57e99c81b9364ca4b92365c361334049', creation_time=1744131950865, experiment_id='2', last_update_time=1744131950865, lifecycle_stage='active', name='my first experimnet', tags={}>

In [108]:
X_train.shape

(1168, 75)

In [109]:
mlflow.start_run(run_name=run_name)
mlflow.log_param("model_type", "Ridge")
mlflow.log_param("random_state", 42)
mlflow.log_param("N-features", 75)


mlflow.log_metrics({
    'test_r2': r2,
    'test_rmse': rmse,
    'test_mae': mae
})

r2 = r2_score(Y_train, prob_train)
rmse = mean_squared_error(Y_train, prob_train, squared=False)
mae = mean_absolute_error(Y_train, prob_train)

mlflow.log_metrics({
    'train_r2': r2,
    'train_rmse': rmse,
    'train_mae': mae
})

mlflow.sklearn.log_model(model, "Ridge")

mlflow.end_run()

2025/04/10 17:20:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Ridge with cleaned features at: https://dagshub.com/Givi-Modebadze/my-first-repo.mlflow/#/experiments/2/runs/f4da5e59f3be4318aa4aeb40a75d7af6
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/my-first-repo.mlflow/#/experiments/2
